In [1]:
import random

# List of stars and planets
stars = [
    "Sirius", "Canopus", "Rigil Kentaurus", "Arcturus", "Vega", "Capella", "Rigel",
    "Procyon", "Achernar", "Betelgeuse", "Hadar", "Altair", "Aldebaran", "Spica", 
    "Antares", "Pollux", "Fomalhaut", "Deneb", "Mimosa", "Regulus", "Adhara", 
    "Shaula", "Castor", "Gacrux", "Bellatrix", "Alnilam", "Alnair", "Alnitak", 
    "Eta Carinae", "Alphard", "Algol", "Dubhe", "Wezen", "Sargas", "Alhena", 
    "Kaus Australis", "Peacock", "Atria", "Markab", "Alcor", "Menkalinan", "Mirfak",
    "Gienah", "Acrux", "Alcyone", "Pherkad", "Maia", "Menkent", "Merak", "Dschubba", 
    "Zeta Reticuli", "Izar", "Zubenelgenubi", "Denebola", "Mintaka", "Caph", "Arctans",
    "Enif", "Ruchbah", "Shedar", "Scheat", "Sualocin", "Tarazed", "Ankaa", "Nunki",
    "Mirzam", "Saiph", "Hamal", "Acamar", "Phecda", "Baten Kaitos", "Almach", "Kochab",
    "Navi", "Unukalhai", "Sabik", "Tiaki", "Furud", "Aldhibah", "Mira", "Castula",
    "Tureis", "Maia", "Merope", "Sterope", "Atlas", "Pleione", "Electra", "Celaeno", 
    "Taygeta", "Alpheratz", "Rasalhague", "Rastaban", "Muphrid", "Kornephoros", 
    "Sadalmelik", "Venus", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto",
    "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9", "M10", "M11", "M12", "M13",
    "M14", "M15", "M16", "M17", "M18", "M19", "M20", "M21", "M22", "M23", "M24",
    "M25", "M26", "M27", "M28", "M29", "M30", "M31", "M32", "M33", "M34", "M35",
    "M36", "M37", "M38", "M39", "M40", "M41", "M42", "M43", "M44", "M45", "M46",
    "M47", "M48", "M49", "M50", "M51", "M52", "M53", "M54", "M55", "M56", "M57",
    "M58", "M59", "M60", "M61", "M62", "M63", "M64", "M65", "M66", "M67", "M68",
    "M69", "M70", "M71", "M72", "M73", "M74", "M75", "M76", "M77", "M78", "M79",
    "M80", "M81", "M82", "M83", "M84", "M85", "M86", "M87", "M88", "M89", "M90",
    "M91", "M92", "M93", "M94", "M95", "M96", "M97", "M98", "M99", "M100", "M101",
    "M102", "M103", "M104", "M105", "M106", "M107", "M108", "M109", "M110", "NGC 1","NGC 2","NGC 4","NGC 5"
]

# List of sentence templates
sentence_templates = [
    "Point the telescope at {}.",
    "Show me {} in the sky.",
    "Where is {} located?",
    "Find the position of {}.",
    "What are the coordinates of {}?",
    "{} is one of the brightest stars.",
    "Can you locate {}?",
    "Is {} visible tonight?",
    "We need to find {}.",
    "The star {} is quite famous.",
    "Astronomers often observe {}."
]

# Function to generate training data with lowercase augmentation
def generate_training_data(star_list, template_list):
    train_data = []
    for star in star_list:
        # Create both title case and lowercase versions of the sentences
        for case_type in [star, star.lower()]:
            sentence_template = random.choice(template_list)
            sentence = sentence_template.format(case_type)
            start_idx = sentence.index(case_type)
            end_idx = start_idx + len(case_type)
            train_data.append((sentence, {"entities": [(start_idx, end_idx, "CELESTIAL_OBJECT")]}))
    return train_data

# Generate the training data with lowercase augmentation
train_data = generate_training_data(stars, sentence_templates)

# Check the first 5 examples
for entry in train_data[:5]:
    print(entry)


('Find the position of Sirius.', {'entities': [(21, 27, 'CELESTIAL_OBJECT')]})
('The star sirius is quite famous.', {'entities': [(9, 15, 'CELESTIAL_OBJECT')]})
('Point the telescope at Canopus.', {'entities': [(23, 30, 'CELESTIAL_OBJECT')]})
('What are the coordinates of canopus?', {'entities': [(28, 35, 'CELESTIAL_OBJECT')]})
('Show me Rigil Kentaurus in the sky.', {'entities': [(8, 23, 'CELESTIAL_OBJECT')]})


In [2]:
import pickle

# Save the training data to a pickle file
with open("train_data.pkl", "wb") as f:
    pickle.dump(train_data, f)


In [13]:
import spacy
from spacy.training import Example

# Load the small English model
nlp = spacy.load("en_core_web_sm")

# Get the pipeline component (NER)
ner = nlp.get_pipe("ner")


In [14]:
ner.add_label("CELESTIAL_OBJECT")


1

In [15]:
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]


In [16]:
examples = []
for text, annotations in train_data:
    doc = nlp.make_doc(text)
    example = Example.from_dict(doc, annotations)
    examples.append(example)


In [17]:
import random
from spacy.util import minibatch, compounding

# Start training
with nlp.disable_pipes(*unaffected_pipes):  # Only train NER
    optimizer = nlp.resume_training()
    for itn in range(21):  # Number of iterations
        random.shuffle(examples)
        losses = {}
        batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, drop=0.5, losses=losses)
        print(f"Iteration {itn}, Losses: {losses}")


Iteration 0, Losses: {'ner': np.float32(402.44385)}
Iteration 1, Losses: {'ner': np.float32(76.43892)}
Iteration 2, Losses: {'ner': np.float32(60.63463)}
Iteration 3, Losses: {'ner': np.float32(49.31287)}
Iteration 4, Losses: {'ner': np.float32(25.782892)}
Iteration 5, Losses: {'ner': np.float32(6.3486433)}
Iteration 6, Losses: {'ner': np.float32(2.4234555)}
Iteration 7, Losses: {'ner': np.float32(4.776085)}
Iteration 8, Losses: {'ner': np.float32(0.957246)}
Iteration 9, Losses: {'ner': np.float32(9.163038)}
Iteration 10, Losses: {'ner': np.float32(4.251299)}
Iteration 11, Losses: {'ner': np.float32(1.3565184)}
Iteration 12, Losses: {'ner': np.float32(1.7081311)}
Iteration 13, Losses: {'ner': np.float32(3.484655)}
Iteration 14, Losses: {'ner': np.float32(0.67460865)}
Iteration 15, Losses: {'ner': np.float32(2.105983)}
Iteration 16, Losses: {'ner': np.float32(0.00012206755)}
Iteration 17, Losses: {'ner': np.float32(0.004250537)}
Iteration 18, Losses: {'ner': np.float32(0.14419526)}
Iter

In [19]:
# Test the custom model
doc = nlp("NGC 13 is a red supergiant star.")
for ent in doc.ents:
    print(ent.text, ent.label_)


NGC 13 CELESTIAL_OBJECT
star CELESTIAL_OBJECT


In [20]:
# Save the model
nlp.to_disk("custom_celestial_ner")

# Load the saved model later
nlp_custom = spacy.load("custom_celestial_ner")
